In [1]:
import selenium
from selenium import webdriver

import time
import re
import pandas as pd

def to_int(text):
    p = re.compile(r'[ㄱ-힣,]+')
    return int(re.sub(p,"",text))

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import urllib
import numpy as np
import requests

def amos(file_name,col_name):
    amos_product = pd.read_excel(file_name)
    amos_list = amos_product[col_name].to_list()

    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
    # options.add_argument('headless')
    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = options)

    url = 'https://smartstore.naver.com/amoskorea'
    driver.get(url)

    df_total = pd.DataFrame()
    img_url = []

    for i in amos_list:
        time.sleep(2)
        driver.find_element_by_css_selector("div > input._38DpYZHzTy").send_keys(str(i))
        time.sleep(1)
        driver.find_element_by_css_selector("button > span.Y4Wg6LWa6z").click()
        time.sleep(2)
        driver.find_elements_by_css_selector("strong._1Zvjahn0GA")[0].click()
        time.sleep(2)

        detail_img = []
        while True:
            body = driver.find_element_by_css_selector('body')
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(2)
            if len(driver.find_elements_by_css_selector("div._9F9CWn02VE img") ) > 0:
                if 'https' in driver.find_element_by_css_selector("div._9F9CWn02VE img").get_attribute("src"):
                    # 이미지 요소 가져오기
                    imgs = driver.find_elements_by_css_selector("div._9F9CWn02VE img")
                    for img in imgs :
                        x = img.get_attribute("src")
                        detail_img.append(x)
                    
                    if len([s for s in detail_img if "data" in s]) > 0:
                        detail_img=[]
                        for j in range(10):
                            body.send_keys(Keys.PAGE_DOWN)
                            
                        imgs2 = driver.find_elements_by_css_selector("div._9F9CWn02VE img")
                        for img2 in imgs2:
                            x = img2.get_attribute("src")
                            detail_img.append(x)
                    break

        df = pd.DataFrame(detail_img).T
        df['title'] = np.nan
        df['title'] = i
        df['img_url'] = driver.find_element_by_css_selector("div._23RpOU6xpc > img").get_attribute("src")

        df_total = pd.concat([df,df_total],axis=0)

    amos_list = df_total['title'].to_list()
    price_1 = []
    price_2 = []
    price_3 = []
    price_4 = []
    price_5 = []
    for i in amos_list:
        url_lowest = 'https://search.shopping.naver.com/search/all?query='+str(i)+'&cat_id=&frm=NVSHATC'
        headers = {'users':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36'}
        time.sleep(2)

        response = requests.get(url_lowest,headers=headers)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        try:
            p1 = to_int(soup.select('span.price_num__2WUXn')[0].get_text())
            price_1.append(p1)
        except:
            price_1.append(np.nan)

        try:
            p2 = to_int(soup.select('span.price_num__2WUXn')[1].get_text())
            price_2.append(p2)
        except:
            price_2.append(np.nan)

        try:
            p3 = to_int(soup.select('span.price_num__2WUXn')[2].get_text())
            price_3.append(p3)
        except:
            price_3.append(np.nan)

        try:
            p4 = to_int(soup.select('span.price_num__2WUXn')[3].get_text())
            price_4.append(p4)
        except:
            price_4.append(np.nan)
        
        try:
            p5 = to_int(soup.select('span.price_num__2WUXn')[4].get_text())
            price_5.append(p5)
        except:
            price_5.append(np.nan)

    df_total['price_1'] = price_1
    df_total['price_2'] = price_2
    df_total['price_3'] = price_3
    df_total['price_4'] = price_4
    df_total['price_5'] = price_5

    df_total = df_total[['title','img_url',0,1,2,3,4,5,6,7,8,'price_1','price_2','price_3','price_4','price_5']]
    df_total = df_total.rename(columns={0:'detail_img_1',1:'detail_img_2',2:'detail_img_3',3:'detail_img_4',4:'detail_img_5',5:'detail_img_6',6:'detail_img_7',7:'detail_img_8',8:'detail_img_9'}) 

    return df_total.to_excel("amos.xlsx",index=False)

In [ ]:
# 사용법: ecommerce('파일이름 + 확장자', '열 이름')
amos("amos_product_list.xlsx","상품명\n[필수]")